![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [73]:
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.utils import serialize_keras_object
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import json
from datetime import datetime

In [74]:
start = datetime(2020, 1, 1)
end = datetime(2022, 1, 1)

In [75]:
qb = QuantBook()
symbol = qb.add_crypto("BTCUSD", Resolution.DAILY).symbol
history = qb.history(symbol, start, end)

In [76]:
history.head()

In [77]:
# data categories used to predict price movement
daily_pct_change = history[["open", "high", "low", "close", "volume"]].pct_change().dropna()
df = daily_pct_change

df.head()

In [78]:
indexes = df[((df.volume == float("inf")))].index

for i in indexes:
    df.at[i, "volume"] = max(df.volume.drop(indexes))


In [79]:
# set up features and labels to predict binary (up/down) price movement
n_steps = 30
features = []
labels = []
for i in range(len(df)-n_steps):
    input_data = df.iloc[i:i+n_steps].values
    features.append(input_data)
    if df['close'].iloc[i+n_steps] >= 0:
        label = 1
    else:
        label = 0

    labels.append(label)

In [80]:
features = np.array(features)
labels = np.array(labels)

In [81]:
# split into 70/30 train/test data

test_length = int(len(features)*0.3)

X_train = features[test_length:]
X_test = features[:test_length]

Y_train = labels[test_length:]
Y_test = labels[:test_length]

# ratio of up to down days (tested to ensure it's around 0.5)
sum(Y_train)/len(Y_train)

In [82]:
# initialize neural net layers
model = Sequential([Dense(30, input_shape = X_train[0].shape, activation = 'relu'), 
                    Dense(20, activation = "relu"), Flatten(), Dense(1, activation = 'sigmoid')])

In [83]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy', 'mse'])


In [84]:
# epochs optimized for training/testing accuracy score (below)
model.fit(X_train, Y_train, epochs = 10)

In [85]:
# plot test data performance

Y_hat = model.predict(X_test)

results = pd.DataFrame({'Y': Y_test.flatten(), 'Y_hat': Y_hat.flatten()})

results.plot(title = "Model performance (predicted vs actual)", figsize = (17, 7))

In [86]:
# Accuracy score for training data

pred_train = model.predict(X_train)

scores = model.evaluate(X_train, Y_train, verbose = 0)

print("Training accuracy:", scores[1], "\nTraining error:", (1-scores[1]))

In [87]:
# Accuracy score for testing data

pred_test = model.predict(X_test)

scores = model.evaluate(X_test, Y_test, verbose = 0)

print("Testing accuracy:", scores[1], "\nTesting error:", (1-scores[1]))

In [88]:
# saving the model into the quantbook wrapper class

model_str = json.dumps(serialize_keras_object(model))

model_key = 'bitcoin_price_predictor'

qb.ObjectStore.Save(model_key, model_str)

In [89]:
if qb.ObjectStore.ContainsKey(model_key):
    model_str = qb.ObjectStore.Read(model_key)
    config = json.loads(model_str)['config']
    model = Sequential.from_config(config)

In [90]:
# sample usage of the model for the current date

testDate = datetime.now()

df = qb.History(symbol, testDate - timedelta(40), testDate).loc[symbol]

df_change = df[["open", "high", "low", "close", "volume"]].pct_change().dropna()

model_input = []

for index, row in df_change.tail(30).iterrows():
    model_input.append(np.array(row))

model_input = np.array([model_input])

if round(model.predict(model_input)[0][0]) == 0:
    print("down")
else:
    print("up")